# Azure Search X Langchain

In [1]:
import streamlit as st
import openai
# used to create the memory
from langchain.memory import ConversationBufferMemory
# used to create the retriever
from langchain.retrievers import AzureCognitiveSearchRetriever
# used to create the retrieval tool
from langchain.agents.agent_toolkits import create_retriever_tool
# used to create the prompt template
from langchain.agents.openai_functions_agent.base import OpenAIFunctionsAgent
from langchain.schema import SystemMessage
from langchain.prompts import MessagesPlaceholder
# used to create the agent executor
from langchain.chat_models import ChatOpenAI
from langchain.agents import AgentExecutor
# To load environment variables
from dotenv import load_dotenv, find_dotenv
import os

In [9]:
_ = load_dotenv(find_dotenv())
# Initialize the memory
# This is needed for both the memory and the prompt
memory_key = "history"

memory = ConversationBufferMemory(memory_key=memory_key, return_messages=True)

messages = [
        {"role": "assistant", "content": "Welcome! Let's get ready to learn!"}
    ]

def get_retriever():
    return AzureCognitiveSearchRetriever(
        content_key="content",
        top_k=3,
        index_name="test1",
        api_key=os.getenv("AZURE_AI_SEARCH_API_KEY"),
        service_name="sb-search",
    )

retriever = get_retriever()

# define the retriever tool
tool = create_retriever_tool(
    retriever=retriever,
    name="search_class_materials",
    description="Search the class materials for information relevant to the question, and return the most relevant documents and metadata.",
)

tools = [tool]

# define the prompt
system_message = SystemMessage(
        content=("""You are a knowledgeable tutor to guide students:
                - You have tools letting you search the class materials and return relevant documents. Use them to answer questions about class materials.
                - When a question is about class material, guide students to this material for a more in-depth understanding including citations if possible
                - For questions outside class materials, guide them toward discovering the answer.
                - If you don't know an answer, refer them to their teacher.
                Your role is not to answer questions, but to guide students towards learning how to find answers themselves.
"""
        )
)
prompt_template = OpenAIFunctionsAgent.create_prompt(
        system_message=system_message,
        extra_prompt_messages=[MessagesPlaceholder(variable_name=memory_key)]
    )

# Initialize the LLM
llm = ChatOpenAI(api_key=os.getenv("OPENAI_API_KEY"),
                 )

# instantiate agent
agent = OpenAIFunctionsAgent(llm=llm, tools=tools, prompt=prompt_template)
agent_executor = AgentExecutor(agent=agent, tools=tools, memory=memory, verbose=True)

In [10]:
# Prompt for user input and display message history
prompt = 'What class materials are uploaded?'

messages.append({"role": "user", "content": prompt})

# Pass query to chat engine and display response
# If last message is not from assistant, generate a new response
if messages[-1]["role"] != "assistant":
    response = agent_executor({"input": prompt})
    message = {"role": "assistant", "content": response["output"]}
    messages.append(message) # Add response to message history



> Entering new AgentExecutor chain...

Invoking: `search_class_materials` with `{'query': 'class materials'}`


[Document(page_content='DAY 10: MBA MINICOURSES Borden, Neil H., Jr. “Class Notes on Conducting International Busine ss,” Darden Graduate School of Business, Charlottesville, V irginia, used with per mission.', metadata={'@search.score': 5.8151445, 'id': 'cd00464e9ff9c6e721728576cc80aa9fc5358e6479d4e3c71f0172d7db902ebf', 'title': 'Lesson - 5: Ten Day MBA', 'filename': 'C:\\Users\\ammar\\Downloads\\Steven A. Silbiger - The Ten-Day MBA-Harper Business (2012).pdf', 'page_number': 516, 'type': 'CLASS_MATERIAL', 'title_vector': [-0.0129780285, -0.009683233, 0.008913291, -0.036263566, -0.0046127145, 0.014982651, -0.036513276, -0.005802309, -0.016078603, -0.025983801, 0.0044115586, 0.021683225, 0.010564157, 0.007165314, 0.0038878592, -0.0023497099, 0.021197677, -0.008268204, 0.020712128, 0.0025231203, -0.0024485537, -0.0011566469, -0.02254334, 0.0066173375, 0.0013473982, -0.019338

BadRequestError: Error code: 400 - {'error': {'message': "This model's maximum context length is 4097 tokens. However, your messages resulted in 67929 tokens (67869 in the messages, 60 in the functions). Please reduce the length of the messages or functions.", 'type': 'invalid_request_error', 'param': 'messages', 'code': 'context_length_exceeded'}}